# Kiểm tra readme trước

# Thuật toán 1: SKYFUP

In [3]:
import sys, os
import time
import tracemalloc

## Cài đặt cấu trúc dữ liệu

### Cài đặt UP-Lists

In [75]:
class Up:
    """
    Đối tượng lưu trữ các thông tin của i ứng với giao dịch (TID, xác xuất, tiện ích, tiện ích còn lại, tiện ích giao dịch)
    """
    def __init__(self, TID, prob, util, r_utility, trans_util):
        """
        Khởi tạo UP

        Args:
            TID (int): mã giao dịch
            prob (float): xác xuất
            util (int): giá trị tiện ích
            r_utility (int): tiện ích còn lại
            trans_util (int): tiện ích giao dịch
        """
        self.TID = TID
        self.prob = round(prob, 2)
        self.util = util
        self.r_utility = r_utility
        self.trans_util = trans_util

    def combine_with(self, other_up):
        """
        Kết hợp 2 Up với nhau

        Args:
            other_up (Up): Up cần kết hợp

        Returns:
            Up: một Up mới
        """
        combined_prob = round(self.prob * other_up.prob, 2)
        combined_util = self.util + other_up.util
        return Up(self.TID, combined_prob, combined_util, other_up.r_utility, other_up.trans_util)

    def __repr__(self):
        return f"Up(TID={self.TID}, prob={self.prob}, util={self.util}, r_utility={self.r_utility}, trans_util={self.trans_util})\n"

### Cài đặt IMCUP-Lists

In [74]:
class IMCUP:
    """
    Đối tượng lưu trữ các thông tin của một item bao gồm: tên, độ hỗ trợ mong đợi, danh sách Up,
        giá trị xác xuất lớn nhất trong Up, TWU, giá trị tiện ích, phần tử cuối của item
    """
    def __init__(self, name, up_list=None):
        """
        Khởi tạo IMCUP

        Args:
            name (str): tên của IMCUP hay tên của item
            up_list (List[Up], None): danh sách Up của item
        """
        self.name = name
        self.exp_sup = round(sum(up.prob for up in up_list), 2) if up_list else 0
        self.up_list = up_list if up_list else []
        self.max_prob = round(max(up.prob for up in up_list), 2) if up_list else 0
        self.trans_wei_util = sum(up.trans_util for up in up_list) if up_list else 0
        self.utility = sum(up.util for up in up_list) if up_list else 0
        self.r_utility = sum(up.r_utility for up in up_list) if up_list else 0
        self.last = []

    def update(self, probability: float, TID, trans_util, r_utility, util_value) -> None:
        """
        Cập nhật lại các giá trị trong IMCUP

        Args:
            probability (float): xác xuất tồn tại
            TID (_type_): mã giao dịch
            trans_util (_type_): tiện ích giao dịch
            r_utility (_type_): tiện ích còn lại
            util_value (_type_): giá trị tiện ích
        """
        probability = round(probability, 2)  
        up = Up(TID, probability, util_value, r_utility, trans_util)
        self.exp_sup = round(self.exp_sup + probability, 2) 
        self.utility += util_value
        self.r_utility += r_utility
        self.up_list.append(up)
        self.max_prob = max(self.max_prob, probability)
        self.trans_wei_util += trans_util

    def combine_up(self, up_list_x, tep_list_y):
        """
        Kết hợp hai danh sách Up

        Args:
            up_list_x (List[Up]): danh sách Up của item X
            tep_list_y (List[Up]): danh sách Up của item Y

        Returns:
            List[Up]: danh sách Up của item XY
        """
        up_list_xy = []
        i, j = 0, 0
        while i < len(up_list_x) and j < len(tep_list_y):
            tX = up_list_x[i]
            tY = tep_list_y[j]
            if tX.TID < tY.TID:
                i += 1
            elif tX.TID > tY.TID:
                j += 1
            else:
                combined_prob = round(tX.prob * tY.prob, 2)
                combined_util = tX.util + tY.util
                up_list_xy.append(Up(tX.TID, combined_prob, combined_util, tY.r_utility, tX.trans_util))
                i += 1
                j += 1
        return up_list_xy

    def combine_with(self, other_up):
        """
        Kết hợp hai IMCUP lại với nhau

        Args:
            other_up (IMCUP): IMCUP tham gia kết hợp

        Returns:
            IMCUP: một IMCUP mới
        """
        if len(other_up.last) == 0:
            combined_name = self.name + ', ' + other_up.name
            combined_tep_list = self.combine_up(self.up_list, other_up.up_list)
            last = [other_up]
        else:
            combined_name = self.name + ', ' + other_up.last[0].name
            combined_tep_list = self.combine_up(self.up_list, other_up.last[0].up_list)
            last = other_up.last

        combined_cup = IMCUP(combined_name, combined_tep_list)
        combined_cup.last = last
        return combined_cup

    def __repr__(self) -> str:
        return f"IMCUP(name={self.name}, exp_sup={self.exp_sup}, r_utility={self.r_utility}, utility={self.utility})"

## Cài đặt thuật toán

In [79]:
class AlgorithmTUHUFP:
    """
    Triển khai thuật toán khai thác top-k mẫu phổ biến tiện ích cao không chắc chắn
    """
    def __init__(self):
        self.start_timestamp = 0
        self.end_timestamp = 0
        self.database_size = 0
        self.database_util = 0
        self.candidates = 0
        self.top_UHUFP = []
        self.single_imcup = {}
        self.threshold = float('-inf')
        self.min_util = float('-inf')
        self.peak_memory_usage = 0

    def read_data(self, file_path, percentage, k) -> None:
        """
        Đọc cơ sỡ dữ liệu từ file và xử lý dữ liệu

        Args:
            file_path (str): đường dẫn chứa file cơ sở dữ liệu
            percentage (float): ngưỡng tiện ích (%)
            k (int): số lượng mẫu cần tìm
        """

        file_paths = file_path.split(", ")
 
        try:
            with open(file_paths[0], 'r') as file1, open(file_paths[1], 'r') as file2:
                print("Reading data . . .")
                tlines = file1.readlines()
                ulines = file2.readlines()
                i_name = tlines[0].strip().split(" ")

                TID = 1
                # xử lý từng giao dịch
                for prob_line, util_line in zip(tlines[1:], ulines):
                    self.process_data(i_name, prob_line.strip(), util_line.strip(), TID)
                    TID += 1
                    self.database_size += 1

                self.min_util = int(self.database_util * percentage)
                print(f"Minimum utility set to: {self.min_util}")
        except FileNotFoundError as e:
            print(f"File not found: {e}")
            print("STOP ALGORITHM !!!")
            sys.exit(0)
        except Exception as e:
            print(f"An error occurred: {e}")
            sys.exit(0)

            
    def process_data(self, i_name, prob_line, util_line, TID) -> None:
        """
        Xử lý từng giao dịch và tạo IMCUP

        Args:
            i_name (List[str]): danh sách các item
            prob_line (float): danh sách các xác xuất của item
            util_line (int): danh sách tiện ích của item
            TID (int): mã giao dịch
        """

        prob_list = prob_line.split(" ")
        trans = util_line.split(":")
        item_util = trans[0].split(" ") # Danh sách các mục (items)
        trans_util = int(trans[1]) # Tổng lợi ích của giao dịch (transaction utility)
        util_list = trans[2].split(" ") # Danh sách lợi ích từng mục trong giao dịch

        i_util_list = {i: int(util) for i, util in zip(item_util, util_list)}

        self.database_util += trans_util

        for i, prob in enumerate(prob_list):
            prob = prob.strip()
    
            # Chỉ xử lý nếu giá trị xác suất tồn tại và lớn hơn 0
            if prob and float(prob) > 0:
                try:
                    # Lấy tên mục (item name) từ chỉ mục (index)
                    i = i_name[i]
                    probability = round(float(prob), 2)  # Round to 2 decimal places
                   
                   # Lấy giá trị lợi ích của mục hiện tại
                    util_value = i_util_list[i]
                    imcup_name = i
                    
                    # Xác định vị trí của mục trong danh sách các keys
                    keys = list(i_util_list.keys()) 
                    index = keys.index(imcup_name) 

                    # Tính tổng lợi ích của các mục sau mục hiện tại
                    r = sum(i_util_list[k] for k in keys[index + 1:])

                    #kiểm tra imcup đã được tạo hay chưa
                    if imcup_name in self.single_imcup:
                        self.single_imcup[imcup_name].update(probability, TID, trans_util, r, util_value)
                    else:
                        new_up = Up(TID, probability, util_value, r, trans_util)
                        self.single_imcup[imcup_name] = IMCUP(imcup_name, [new_up])
                except ValueError as ve:
                    print(f"Error processing probability '{prob}': {ve}")
                    continue



    def combine_cup(self, upX, upY):
        """
        Kết hơp hai Up X và Y

        Args:
            upX (Up): Up tham gia kết hợp
            upY (Up): Up tham gia kết hợp

        Returns:
           Up: UpXY
        """
        combined_cup = upX.combine_with(upY)
        combined_cup.exp_sup = round(combined_cup.exp_sup, 2) 
        return combined_cup
    
    

    def check(self, combined, k):
        """
        Kiểm tra mẫu có phải là mẫu tiện ích cao

        Args:
            combined (Up): mẫu cần kiểm tra
           TUHUFP k (int): số lượng mẫu cần tìm
        """

        # kiểm tra và thêm vào top k nếu thỏa điều kiện utility > min_util
        if combined.utility >= self.min_util:
            self.top_UHUFP.append({'name': combined.name, 'exp_sup': combined.exp_sup, 'utility': combined.utility})
            self.top_UHUFP.sort(key=lambda x: x['exp_sup'], reverse=True)
            if len(self.top_UHUFP) > k:
                self.top_UHUFP.pop()
                self.threshold = self.top_UHUFP[-1]['exp_sup']
            # cập nhật lại threshold khi top_UHUFP đã đủ k mẫu
            if len(self.top_UHUFP) == k:
                self.threshold = self.top_UHUFP[-1]['exp_sup']


    def Search(self, currentUp, k):
        """
        Khai thác mẫu từ danh sách ứng viên

        Args:
            currentUp (List[Up]): danh sách ứng viên tham gia
            k (int): số lượng mẫu cần tìm
        """
        if len(currentUp) <= 1:
            # dừng thuật toán khi không còn ứng viên tham gia
            return
        # duyệt từng ứng viên
        for i in range(len(currentUp) - 1):
            # kiểm tra điều kiện tiện ích
            if currentUp[i].utility + currentUp[i].r_utility > self.min_util:
                newCupList = []
                for j in range(i + 1, len(currentUp)):
                    overestimate = currentUp[i].exp_sup * currentUp[j].max_prob
                    if currentUp[j].exp_sup < self.threshold or overestimate < self.threshold:
                        break
                    combined = self.combine_cup(currentUp[i], currentUp[j])
                    # kiểm tra và thêm vào top k nếu thỏa điều kiện
                    if combined.exp_sup > self.threshold and combined.trans_wei_util > self.min_util:
                        self.check(combined, k)
                        # thêm các ứng viên mới
                        if combined.trans_wei_util >= self.min_util:
                            newCupList.append(combined)
                            self.candidates += 1
                # tìm kiếm với danh sách ứng viên mới
                self.Search(newCupList, k)

    def create_candidate(self, min_util, k):
        """
        Tìm top k đầu tiên và các ứng viên

        Args:
            min_util (int): ngưỡng tiện ích
            k (int): số lượng mẫu cần tìm

        Returns:
            List[Up]: danh sách các ứng viên
        """
        candidate_list = []
        imcup = sorted(self.single_imcup.values(), key=lambda x: x.exp_sup, reverse=True)
        # lấy k Up đầu tiên có giá trị expSup lớn nhất
        for up in imcup[:k]:
            if up.trans_wei_util >= min_util:
                candidate_list.append(up)
            if up.utility >= min_util:
                self.top_UHUFP.append({'name': up.name, 'exp_sup': up.exp_sup, 'utility': up.utility})
        return candidate_list


    def run(self, file_path, percentage, k):
        """
        Khởi chạy thuật toán

        Args:
            file_path (str): đường dẫn tới file cơ sở dữ liệu
            percentage (_type_): ngưỡng tiện ích (%)
            k (int): số lượng mẫu cần tìm
        """

        self.start_timestamp = time.time()
        self.candidates = 0
        self.database_size = 0

        # đọc dữ liệu từ file
        self.read_data(file_path, percentage, k)
        print({'database util: ': self.database_util})

        # danh sách ứng viên
        candidate_list = self.create_candidate(self.min_util, k)
        if not self.single_imcup:
            print("No Up List was created. STOP ALGORITHM !!!")
            sys.exit(0)
        self.candidates = len(candidate_list)

        # đặt threhold
        if len(self.top_UHUFP) == k:
            self.threshold = self.top_UHUFP[-1]['exp_sup']

        # Bắt đầu theo dõi bộ nhớ
        tracemalloc.start()
        # bắt đầu tìm kiếm
        self.Search(candidate_list, k)
        print(f"Top {k} CUPs:")
        i = 1
        for Up in self.top_UHUFP[:]:
            print(f" {i}: {Up}")
            i +=1
        # Kết thúc theo dõi bộ nhớ và lấy thông tin
        _, self.peak_memory_usage = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.peak_memory_usage /= 10**6  # Chuyển đổi sang MB

        self.end_timestamp = time.time()

    def print_stats(self, path):
        """
        Thống kê thông tin kết quả chạy thuật toán

        Args:
            path (str): đường dẫn lưu thông tin
        """        

        total_utility = 0
        directory = os.path.dirname(path)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        with open(path, 'w') as output_file:
            output_file.write(f"minUtil: {self.min_util}\n")
            self.top_UHUFP.sort(key=lambda x: x['utility'], reverse=True)
            for t in self.top_UHUFP:
                total_utility += t['utility']
                output_file.write(f"{t['name']}: {t['exp_sup']}: {t['utility']}\n")
            output_file.write("=============  TOP-K UFPs v1.20 - STATS =============\n")
            output_file.write(f" Transactions count from database : {self.database_size}\n")
            output_file.write(f" Candidates count : {self.candidates}\n")
            output_file.write(f" Sum utility : {total_utility}\n")
            output_file.write(f" Algorithm run time : {self.end_timestamp - self.start_timestamp:.0f} seconds\n")
            output_file.write(f" Peak memory usage : {self.peak_memory_usage:.2f} MB\n")  # Ghi lại peak memory usage


In [13]:
# a b c d e
# 0.6 0 0.8 0.5 0
# 0 0.7 0.4 0 0
# 0.6 0.6 0.9 0.8 0
# 0 0.5 0 0.8 0
# 0.9 0.7 0.9 0.6 0.8
# 0 0 0.9 0 0.8
# 0 0 0.4 0.9 0
# 0.6 0.8 0 0.8 0.5
# 0.6 0 0.5 0.3 0
# 0 0 0.6 0 0.7


# a d c:65:14 7 44
# b c:37:4 33
# a b d c:38:21 4 2 11
# b d:11:8 3
# e a b d c:49:9 7 6 5 22 
# e c:58:36 22
# d c:23:1 22
# e a b d:61:36 21 2 2
# a d c:59:14 1 44
# e c:42:9 33


In [81]:
# chạy với retail
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/example.txt, ./data/example_utility.txt", 0.3, 6)
algorithm.print_stats("./out/TUHUFP/output_example.txt")

Reading data . . .
Minimum utility set to: 30
{'database util: ': 102}
Top 6 CUPs:
 1: {'name': 'd', 'exp_sup': 3.3, 'utility': 30}
 2: {'name': 'c, d', 'exp_sup': 1.95, 'utility': 38}
 3: {'name': 'd, a', 'exp_sup': 1.2, 'utility': 35}
 4: {'name': 'c, d, a', 'exp_sup': 0.98, 'utility': 42}
 5: {'name': 'c, d, b', 'exp_sup': 0.88, 'utility': 40}
 6: {'name': 'd, a, b', 'exp_sup': 0.55, 'utility': 33}


## Hiện thức foodmart

In [64]:
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_foodmart.txt, ./data/foodmart_utility.txt", 0.0004, 100)
algorithm.print_stats("./out/TUHUFP/foodmart/output_foodmart_top_100.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 100 CUPs:
 1: {'name': '304', 'exp_sup': 14.23, 'utility': 13607}
 2: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 3: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 4: {'name': '1293', 'exp_sup': 12.66, 'utility': 8352}
 5: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 6: {'name': '545', 'exp_sup': 12.1, 'utility': 9419}
 7: {'name': '357', 'exp_sup': 11.97, 'utility': 11520}
 8: {'name': '634', 'exp_sup': 11.88, 'utility': 11881}
 9: {'name': '617', 'exp_sup': 11.75, 'utility': 7527}
 10: {'name': '1423', 'exp_sup': 11.75, 'utility': 16116}
 11: {'name': '916', 'exp_sup': 11.68, 'utility': 15163}
 12: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 13: {'name': '903', 'exp_sup': 11.62, 'utility': 14616}
 14: {'name': '831', 'exp_sup': 11.39, 'utility': 8848}
 15: {'name': '888', 'exp_sup': 11.38, 'utility': 5280}
 16: {'name': '918', 'exp_sup': 11.38, 'utility': 18414}
 17: {'

In [65]:
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_foodmart.txt, ./data/foodmart_utility.txt", 0.0004, 300)
algorithm.print_stats("./out/TUHUFP/foodmart/output_foodmart_top_300.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 300 CUPs:
 1: {'name': '304', 'exp_sup': 14.23, 'utility': 13607}
 2: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 3: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 4: {'name': '1293', 'exp_sup': 12.66, 'utility': 8352}
 5: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 6: {'name': '545', 'exp_sup': 12.1, 'utility': 9419}
 7: {'name': '357', 'exp_sup': 11.97, 'utility': 11520}
 8: {'name': '634', 'exp_sup': 11.88, 'utility': 11881}
 9: {'name': '617', 'exp_sup': 11.75, 'utility': 7527}
 10: {'name': '1423', 'exp_sup': 11.75, 'utility': 16116}
 11: {'name': '916', 'exp_sup': 11.68, 'utility': 15163}
 12: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 13: {'name': '903', 'exp_sup': 11.62, 'utility': 14616}
 14: {'name': '831', 'exp_sup': 11.39, 'utility': 8848}
 15: {'name': '888', 'exp_sup': 11.38, 'utility': 5280}
 16: {'name': '918', 'exp_sup': 11.38, 'utility': 18414}
 17: {'

In [66]:
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_foodmart.txt, ./data/foodmart_utility.txt", 0.0004, 500)
algorithm.print_stats("./out/TUHUFP/foodmart/output_foodmart_top_500.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 500 CUPs:
 1: {'name': '304', 'exp_sup': 14.23, 'utility': 13607}
 2: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 3: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 4: {'name': '1293', 'exp_sup': 12.66, 'utility': 8352}
 5: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 6: {'name': '545', 'exp_sup': 12.1, 'utility': 9419}
 7: {'name': '357', 'exp_sup': 11.97, 'utility': 11520}
 8: {'name': '634', 'exp_sup': 11.88, 'utility': 11881}
 9: {'name': '617', 'exp_sup': 11.75, 'utility': 7527}
 10: {'name': '1423', 'exp_sup': 11.75, 'utility': 16116}
 11: {'name': '916', 'exp_sup': 11.68, 'utility': 15163}
 12: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 13: {'name': '903', 'exp_sup': 11.62, 'utility': 14616}
 14: {'name': '831', 'exp_sup': 11.39, 'utility': 8848}
 15: {'name': '888', 'exp_sup': 11.38, 'utility': 5280}
 16: {'name': '918', 'exp_sup': 11.38, 'utility': 18414}
 17: {'

In [67]:
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_foodmart.txt, ./data/foodmart_utility.txt", 0.0004, 700)
algorithm.print_stats("./out/TUHUFP/foodmart/output_foodmart_top_700.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 700 CUPs:
 1: {'name': '304', 'exp_sup': 14.23, 'utility': 13607}
 2: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 3: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 4: {'name': '1293', 'exp_sup': 12.66, 'utility': 8352}
 5: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 6: {'name': '545', 'exp_sup': 12.1, 'utility': 9419}
 7: {'name': '357', 'exp_sup': 11.97, 'utility': 11520}
 8: {'name': '634', 'exp_sup': 11.88, 'utility': 11881}
 9: {'name': '617', 'exp_sup': 11.75, 'utility': 7527}
 10: {'name': '1423', 'exp_sup': 11.75, 'utility': 16116}
 11: {'name': '916', 'exp_sup': 11.68, 'utility': 15163}
 12: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 13: {'name': '903', 'exp_sup': 11.62, 'utility': 14616}
 14: {'name': '831', 'exp_sup': 11.39, 'utility': 8848}
 15: {'name': '888', 'exp_sup': 11.38, 'utility': 5280}
 16: {'name': '918', 'exp_sup': 11.38, 'utility': 18414}
 17: {'

In [68]:
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_foodmart.txt, ./data/foodmart_utility.txt", 0.0004, 900)
algorithm.print_stats("./out/TUHUFP/foodmart/output_foodmart_top_900.txt")

Reading data . . .
Minimum utility set to: 4804
{'database util: ': 12011023}
Top 900 CUPs:
 1: {'name': '304', 'exp_sup': 14.23, 'utility': 13607}
 2: {'name': '1373', 'exp_sup': 13.55, 'utility': 25560}
 3: {'name': '272', 'exp_sup': 13.54, 'utility': 22678}
 4: {'name': '1293', 'exp_sup': 12.66, 'utility': 8352}
 5: {'name': '1292', 'exp_sup': 12.6, 'utility': 24642}
 6: {'name': '545', 'exp_sup': 12.1, 'utility': 9419}
 7: {'name': '357', 'exp_sup': 11.97, 'utility': 11520}
 8: {'name': '634', 'exp_sup': 11.88, 'utility': 11881}
 9: {'name': '617', 'exp_sup': 11.75, 'utility': 7527}
 10: {'name': '1423', 'exp_sup': 11.75, 'utility': 16116}
 11: {'name': '916', 'exp_sup': 11.68, 'utility': 15163}
 12: {'name': '1012', 'exp_sup': 11.65, 'utility': 19861}
 13: {'name': '903', 'exp_sup': 11.62, 'utility': 14616}
 14: {'name': '831', 'exp_sup': 11.39, 'utility': 8848}
 15: {'name': '888', 'exp_sup': 11.38, 'utility': 5280}
 16: {'name': '918', 'exp_sup': 11.38, 'utility': 18414}
 17: {'

## Hiện thực Chess

In [69]:
# chạy với chess
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_chess.txt, ./data/chess_utility.txt", 0.001, 100)
algorithm.print_stats("./out/TUHUFP/chess/output_chess_top_100.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 100 CUPs:
 1: {'name': '52', 'exp_sup': 1616.54, 'utility': 104178}
 2: {'name': '29', 'exp_sup': 1598.92, 'utility': 70088}
 3: {'name': '58', 'exp_sup': 1595.58, 'utility': 17462}
 4: {'name': '40', 'exp_sup': 1579.27, 'utility': 188760}
 5: {'name': '60', 'exp_sup': 1565.64, 'utility': 68908}
 6: {'name': '62', 'exp_sup': 1537.33, 'utility': 16784}
 7: {'name': '7', 'exp_sup': 1535.24, 'utility': 50790}
 8: {'name': '36', 'exp_sup': 1530.49, 'utility': 16934}
 9: {'name': '56', 'exp_sup': 1513.93, 'utility': 66780}
 10: {'name': '48', 'exp_sup': 1500.86, 'utility': 115367}
 11: {'name': '66', 'exp_sup': 1498.92, 'utility': 66644}
 12: {'name': '34', 'exp_sup': 1493.64, 'utility': 99330}
 13: {'name': '5', 'exp_sup': 1466.73, 'utility': 48369}
 14: {'name': '9', 'exp_sup': 1446.46, 'utility': 31818}
 15: {'name': '25', 'exp_sup': 1445.46, 'utility': 15465}
 16: {'name': '3', 'exp_sup': 1407.44, 'utility'

In [70]:
# chạy với chess
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_chess.txt, ./data/chess_utility.txt", 0.001, 300)
algorithm.print_stats("./out/TUHUFP/chess/output_chess_top_300.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 300 CUPs:
 1: {'name': '52', 'exp_sup': 1616.54, 'utility': 104178}
 2: {'name': '29', 'exp_sup': 1598.92, 'utility': 70088}
 3: {'name': '58', 'exp_sup': 1595.58, 'utility': 17462}
 4: {'name': '40', 'exp_sup': 1579.27, 'utility': 188760}
 5: {'name': '60', 'exp_sup': 1565.64, 'utility': 68908}
 6: {'name': '62', 'exp_sup': 1537.33, 'utility': 16784}
 7: {'name': '7', 'exp_sup': 1535.24, 'utility': 50790}
 8: {'name': '36', 'exp_sup': 1530.49, 'utility': 16934}
 9: {'name': '56', 'exp_sup': 1513.93, 'utility': 66780}
 10: {'name': '48', 'exp_sup': 1500.86, 'utility': 115367}
 11: {'name': '66', 'exp_sup': 1498.92, 'utility': 66644}
 12: {'name': '34', 'exp_sup': 1493.64, 'utility': 99330}
 13: {'name': '5', 'exp_sup': 1466.73, 'utility': 48369}
 14: {'name': '9', 'exp_sup': 1446.46, 'utility': 31818}
 15: {'name': '25', 'exp_sup': 1445.46, 'utility': 15465}
 16: {'name': '3', 'exp_sup': 1407.44, 'utility'

In [71]:
# chạy với chess
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_chess.txt, ./data/chess_utility.txt", 0.001, 500)
algorithm.print_stats("./out/TUHUFP/chess/output_chess_top_500.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 500 CUPs:
 1: {'name': '52', 'exp_sup': 1616.54, 'utility': 104178}
 2: {'name': '29', 'exp_sup': 1598.92, 'utility': 70088}
 3: {'name': '58', 'exp_sup': 1595.58, 'utility': 17462}
 4: {'name': '40', 'exp_sup': 1579.27, 'utility': 188760}
 5: {'name': '60', 'exp_sup': 1565.64, 'utility': 68908}
 6: {'name': '62', 'exp_sup': 1537.33, 'utility': 16784}
 7: {'name': '7', 'exp_sup': 1535.24, 'utility': 50790}
 8: {'name': '36', 'exp_sup': 1530.49, 'utility': 16934}
 9: {'name': '56', 'exp_sup': 1513.93, 'utility': 66780}
 10: {'name': '48', 'exp_sup': 1500.86, 'utility': 115367}
 11: {'name': '66', 'exp_sup': 1498.92, 'utility': 66644}
 12: {'name': '34', 'exp_sup': 1493.64, 'utility': 99330}
 13: {'name': '5', 'exp_sup': 1466.73, 'utility': 48369}
 14: {'name': '9', 'exp_sup': 1446.46, 'utility': 31818}
 15: {'name': '25', 'exp_sup': 1445.46, 'utility': 15465}
 16: {'name': '3', 'exp_sup': 1407.44, 'utility'

In [72]:
# chạy với chess
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_chess.txt, ./data/chess_utility.txt", 0.001, 700)
algorithm.print_stats("./out/TUHUFP/chess/output_chess_top_700.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 700 CUPs:
 1: {'name': '52', 'exp_sup': 1616.54, 'utility': 104178}
 2: {'name': '29', 'exp_sup': 1598.92, 'utility': 70088}
 3: {'name': '58', 'exp_sup': 1595.58, 'utility': 17462}
 4: {'name': '40', 'exp_sup': 1579.27, 'utility': 188760}
 5: {'name': '60', 'exp_sup': 1565.64, 'utility': 68908}
 6: {'name': '62', 'exp_sup': 1537.33, 'utility': 16784}
 7: {'name': '7', 'exp_sup': 1535.24, 'utility': 50790}
 8: {'name': '36', 'exp_sup': 1530.49, 'utility': 16934}
 9: {'name': '56', 'exp_sup': 1513.93, 'utility': 66780}
 10: {'name': '48', 'exp_sup': 1500.86, 'utility': 115367}
 11: {'name': '66', 'exp_sup': 1498.92, 'utility': 66644}
 12: {'name': '34', 'exp_sup': 1493.64, 'utility': 99330}
 13: {'name': '5', 'exp_sup': 1466.73, 'utility': 48369}
 14: {'name': '9', 'exp_sup': 1446.46, 'utility': 31818}
 15: {'name': '25', 'exp_sup': 1445.46, 'utility': 15465}
 16: {'name': '3', 'exp_sup': 1407.44, 'utility'

In [73]:
# chạy với chess
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_chess.txt, ./data/chess_utility.txt", 0.001, 900)
algorithm.print_stats("./out/TUHUFP/chess/output_chess_top_900.txt")

Reading data . . .
Minimum utility set to: 2156
{'database util: ': 2156659}
Top 900 CUPs:
 1: {'name': '52', 'exp_sup': 1616.54, 'utility': 104178}
 2: {'name': '29', 'exp_sup': 1598.92, 'utility': 70088}
 3: {'name': '58', 'exp_sup': 1595.58, 'utility': 17462}
 4: {'name': '40', 'exp_sup': 1579.27, 'utility': 188760}
 5: {'name': '60', 'exp_sup': 1565.64, 'utility': 68908}
 6: {'name': '62', 'exp_sup': 1537.33, 'utility': 16784}
 7: {'name': '7', 'exp_sup': 1535.24, 'utility': 50790}
 8: {'name': '36', 'exp_sup': 1530.49, 'utility': 16934}
 9: {'name': '56', 'exp_sup': 1513.93, 'utility': 66780}
 10: {'name': '48', 'exp_sup': 1500.86, 'utility': 115367}
 11: {'name': '66', 'exp_sup': 1498.92, 'utility': 66644}
 12: {'name': '34', 'exp_sup': 1493.64, 'utility': 99330}
 13: {'name': '5', 'exp_sup': 1466.73, 'utility': 48369}
 14: {'name': '9', 'exp_sup': 1446.46, 'utility': 31818}
 15: {'name': '25', 'exp_sup': 1445.46, 'utility': 15465}
 16: {'name': '3', 'exp_sup': 1407.44, 'utility'

## Hiện thực retail

In [ ]:
# chạy với retail
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_retail.txt, ./data/retail_utility.txt", 0.0001, 100)
algorithm.print_stats("./out/TUHUFP/retail/output_retail_top_100.txt")

In [ ]:
# chạy với retail
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_retail.txt, ./data/retail_utility.txt", 0.0001, 300)
algorithm.print_stats("./out/TUHUFP/retail/output_retail_top_300.txt")

In [ ]:
# chạy với retail
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_retail.txt, ./data/retail_utility.txt", 0.0001, 500)
algorithm.print_stats("./out/TUHUFP/retail/output_retail_top_500.txt")

In [ ]:
# chạy với retail
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_retail.txt, ./data/retail_utility.txt", 0.0001, 700)
algorithm.print_stats("./out/TUHUFP/retail/output_retail_top_700.txt")

In [ ]:
# chạy với retail
algorithm = AlgorithmTUHUFP()
algorithm.run("./data/input_retail.txt, ./data/retail_utility.txt", 0.0001, 900)
algorithm.print_stats("./out/TUHUFP/retail/output_retail_top_900.txt")

# Thuật toán 2: TUHUFP-Growth

In [82]:
import time, os
import tracemalloc
from typing import Dict, List, Tuple
from collections import defaultdict

## Cài đặt cấu trúc dữ liệu

In [2]:
class Node:
    """
    Lớp đại diện cho một nút trong cây UHUFP (Cây mẫu tần suất tiện ích cao không chắc chắn).

    Thuộc tính:
        item (str hoặc None): Mục được lưu trữ trong nút này. Mặc định là None.
        prob (float): Xác suất liên quan đến nút này. Mặc định là 0.0.
        count (int): Số lần (hỗ trợ) của nút này. Mặc định là 0.
        utility (int): Giá trị tiện ích của mục của nút này. Mặc định là 0.
        nu (int): Tiện ích của nút, là giá trị tiện ích ước tính cho nút này. Mặc định là 0.
        children (dict): Từ điển lưu trữ các nút con, trong đó khóa là mục và giá trị là nút con tương ứng.
        parent (Node hoặc None): Tham chiếu đến nút cha của nút này. Mặc định là None.
    """

    def __init__(self, item=None, prob=0.0, count=0, utility=0, nu=0):
        """
        Khởi tạo một nút mới trong cây UHUFP.
        """
        self.item = item
        self.prob = prob
        self.count = count
        self.utility = utility
        self.children = {}
        self.parent = None
        self.nu = nu

In [3]:
class UHUFP_tree:
    """
    Lớp đại diện cho cây UHUFP (Cây mẫu tần suất tiện ích cao không chắc chắn).

    Thuộc tính:
        root (Node): Gốc của cây, là một đối tượng Node.
        header_table (defaultdict): Bảng tiêu đề, lưu trữ các liên kết đến các nút với cùng một mục.
    """

    def __init__(self):
        """
        Khởi tạo cây UHUFP với gốc là một nút rỗng và bảng tiêu đề là một defaultdict.
        """
        self.root = Node()
        self.header_table = defaultdict(list)

    def insert_reorganized_transaction(self, transaction, utilities, sorted_items):
        """
        Chèn một giao dịch đã được tổ chức lại vào cây UHUFP.

        Tham số:
            transaction (dict): Giao dịch với các mục và xác suất tương ứng.
            utilities (dict): Bảng tiện ích của các mục trong giao dịch.
            sorted_items (list): Danh sách các mục được sắp xếp theo thứ tự TWU giảm dần.
        """
        current_node = self.root
        rtu = sum(utilities.values())  # Tổng tiện ích của giao dịch

        for i, item in enumerate(sorted_items):
            item_prob = transaction[item]
            item_utility = utilities[item]
            
            if (item, item_prob, item_utility) in current_node.children:
                child_node = current_node.children[(item, item_prob, item_utility)]
                child_node.count += 1
                items_after = sorted_items[i+1:]
                remaining_utility = sum(utilities.get(it, 0) for it in items_after)
                child_node.nu += rtu - remaining_utility
            else:
                child_node = Node(item=item, prob=item_prob, count=1, utility=item_utility)
                items_after = sorted_items[i+1:]
                remaining_utility = sum(utilities.get(it, 0) for it in items_after)
                child_node.nu = rtu - remaining_utility
                
                child_node.parent = current_node
                current_node.children[(item, item_prob, item_utility)] = child_node
                self.header_table[item].append(child_node)
            
            current_node = child_node

    def insert_reorganized_path(self, transaction, utilities, sorted_items, miu):
        """
        Chèn một đường dẫn đã được tổ chức lại vào cây UHUFP.

        Tham số:
            transaction (dict): Giao dịch với các mục và xác suất tương ứng.
            utilities (dict): Bảng tiện ích của các mục trong giao dịch.
            sorted_items (list): Danh sách các mục được sắp xếp theo thứ tự tiện ích đường dẫn của item trong CPB giảm dần.
            miu (dict): Bảng tiện ích mục nhỏ nhất cho mỗi mục trong đường dẫn.
        """
        current_node = self.root
        for i, item in enumerate(sorted_items):
            item_prob = transaction[item]
            item_utility = utilities[item]

            if (item, item_prob, item_utility) in current_node.children:
                child_node = current_node.children[(item, item_prob, item_utility)]
                child_node.count += 1
                utility_after = sum(miu.get(x, 0) * child_node.count for x in list(sorted_items.keys())[i + 1:])
                child_node.nu += sorted_items[item] - utility_after
            else:
                child_node = Node(item=item, prob=item_prob, count=1, utility=item_utility)
                utility_after = sum(miu.get(x, 0) * child_node.count for x in list(sorted_items.keys())[i + 1:])
                child_node.nu = sorted_items[item] - utility_after

                child_node.parent = current_node
                current_node.children[(item, item_prob, item_utility)] = child_node
                self.header_table[item].append(child_node)

            current_node = child_node

    def print_header_table(self):
        """
        In bảng tiêu đề (header table) của cây UHUFP.
        """
        print("\nHeader Table:")
        for item, nodes in self.header_table.items():
            for node in nodes:
                print(f"  -> Node: Item={node.item}, Support={node.prob}, Count={node.count}, Utility={node.utility}")
        print("End of Header Table\n")

    def print_tree(self):
        """
        In cấu trúc của cây UHUFP.
        """
        def print_node(node, level=0):
            indent = ' ' * (level * 4)
            print(f"{indent}Item: {node.item}, Support: {node.prob}, Count: {node.count}, Utility: {node.utility}, Nu: {node.nu}")
            for child in node.children.values():
                print_node(child, level + 1)

        print("UHUFP Tree:")
        print_node(self.root)
        print("End of UHUFP Tree")

    def prune_patterns(self, k, item_twu, minUtil):
        """
        Cắt bớt các mẫu không cần thiết dựa trên TWU và số lượng top-k.

        Tham số:
            k (int): Số lượng các mục top-k cần giữ lại.
            item_twu (dict): Bảng TWU của các mục.
            minUtil (int): Ngưỡng tiện ích
        """
        item_exp_sups = {}
        for item, nodes in self.header_table.items():
            exp_sup = sum(node.prob * node.count for node in nodes)
            item_exp_sups[item] = exp_sup
        # Sắp xếp theo expSup của item
        sorted_items_by_exp_sup = sorted(self.header_table.keys(), key=lambda item: item_exp_sups[item], reverse=True)
        self.header_table = {item: self.header_table[item] for item in sorted_items_by_exp_sup}

        filtered_header_table = defaultdict(list)
        for item, nodes in self.header_table.items():
            k -= 1
            twu = item_twu[item]
            if twu >= minUtil:
                filtered_header_table[item] = self.header_table[item]
            if k <= 0:
                break
        self.header_table = filtered_header_table

    def prune_tree(self, node):
        """
        Loại bỏ các nút khỏi cây nếu mục của chúng không có trong bảng tiêu đề.

        Tham số:
            node (Node): Nút hiện tại đang xử lý để cắt tỉa.
        """
        items_to_keep = set(self.header_table.keys())
        children_to_keep = {k: v for k, v in node.children.items() if v.item in items_to_keep}
            
        node.children = children_to_keep
            
        for child in node.children.values():
            self.prune_tree(child)

## Cài đặt thuật toán

In [4]:
class TUHUFP_Growth:
    """Lớp này triển khai thuật toán khai thác top-k mẫu phổ biến tiện ích cao không chắc chắn
    """

    def __init__(self):
        self.start_timestamp = 0
        self.end_timestamp = 0
        self.database_size = 0
        self.peak_memory_usage = 0
        self.top_k_UHUFP = []
        self.minUtil = 0

    
    def calculate_expected_support(self, paths, prefix):
        """Tính độ hỗ trợ mong đợi của mẫu (prefix) cụ thể thông qua đường dẫn đã thu thập

        Args:
            paths (Tuple(Tuple(List[str,float,int,int], int))): đường dẫn của prefix
            prefix (List[str]): mẫu cần tính giá trị tiện ích
        Returns:
            int: giá trị độ hỗ trợ mong đợi của prefix
        """

        total_expected_support = 0.0
        # paths = self.collect_paths_prefix(prefix)

        for path, count in paths:
            # Tạo tập hợp các item trong path
            path_items = {node[0] for node in path}

            # Kiểm tra xem tất cả các item trong prefix có nằm trong path không
            if all(item in path_items for item in prefix):
                path_support = 1.0
                for item in prefix:
                    # Tìm item trong path
                    i = next((i for i in path if i[0] == item), None)
                    if i:
                        path_support = round((path_support*i[1]),2) 
                    else:
                        path_support = 0
                        break

                total_expected_support += path_support * count 

        return round(total_expected_support, 2)




    def calculate_utility(self, paths, prefix):
        """Tính giá trí tiện ích của mẫu (prefix) cụ thể thông qua đường dẫn đã thu thập

        Args:
            paths (Tuple(Tuple(List[str,float,int,int], int))): đường dẫn của prefix
            prefix (List[str]): mẫu cần tính giá trị tiện ích
        Returns:
            int: giá trị tiện ích của prefix
        """

        prefix_utility = 0
        # paths = self.collect_paths_prefix(prefix)

        for path, count in paths:
            # Tạo tập hợp các item trong path
            path_items = {node[0] for node in path}

            # Kiểm tra xem tất cả các item trong prefix có nằm trong path không
            if all(item in path_items for item in prefix):
                sum_utility = 0
                for item in prefix:
                    # Tìm item trong path
                    i = next((i for i in path if i[0] == item), None)
                    if i:
                        sum_utility += i[2] 
                    else:
                        sum_utility = 0
                        break

                prefix_utility += sum_utility * count 

        return prefix_utility

        
 
    def calculate_twu(self, paths, prefix):
        """Tính TWU của prefix 

        Args:
            paths (Tuple(Tuple(List[str,float,int,int], int))): đường dẫn của prefix
            prefix (List[str]): mẫu cần tính TWU
        Returns:
            int: giá trị TWU
        """
        prefix_twu = 0
        # paths = self.collect_paths_prefix(prefix)

        for path, count in paths:
            # Tạo tập hợp các item trong path
            path_items = {node[0] for node in path}

            # Kiểm tra xem tất cả các item trong prefix có nằm trong path không
            if all(item in path_items for item in prefix):
                sum_twu= 0
                # Tính expected support cho path bằng cách nhân các xác suất (prob) của các items trong prefix
                for item in prefix:
                    # Tìm item trong path
                    i = next((i for i in path if i[0] == item), None)
                    if i:
                        sum_twu = i[3]  # Nhân xác suất (prob) của item trong path

                prefix_twu += sum_twu * count  # Nhân với count của path

        return prefix_twu


    def mine_top_k_patterns(self, uhufp_tree, k, minSup, minUtil, expected_support, prefix=None, top_k=None):
        """ Phương thức chính khai thác các tập mẫu UHUFP

        Args:
            uhufp_tree (UHUFP_tree): cây UHUFP của mẫu 
            k (int): số lượng mẫu cần tìm
            minSup (float): ngưỡng phổ biến trong top-k
            minUtil (int): ngưỡng tiện ích
            expected_support (float): giá trị độ hỗ trợ mong đợi của prefix, ban đâu là 0
            prefix (List[str]): là tiền tố của mẫu. Mặc đinh ban đầu None.
            top_k (List[UHUFP], ): danh sách chứa k mẫu UHUFP. Mặc đinh ban đầu None.

        Returns:
            List[UHUFP]: danh sách chứa k mẫu UHUFP.
        """
        if top_k is None:
            top_k = []
        if prefix is None:
            prefix = []
        if len(top_k) == k and top_k is not None:
            minSup = top_k[-1][1]

        #Duyệt từng phần từ trong bảng header
        for item, nodes in uhufp_tree.header_table.items():
            # kiểm tra item khác prefix
            if item not in prefix:
                max_prob = max(node.prob for node in nodes if node.item == item)
                overestimate = max_prob*expected_support
                if overestimate < minSup:
                    return top_k
                # Tạo mẫu mới từ prefix và item
                new_prefix = [item] + prefix

                if any(existing_pattern[0] == tuple(sorted(new_prefix)) for existing_pattern in top_k):
                    continue
                # print(f"ITEM: {new_prefix}")
                # thu thập conditional pattern base của mẫu
                conditional_pattern_base, miu, path_util_item = self.build_conditional_pattern_base(uhufp_tree, new_prefix, minUtil)
                if path_util_item >= minUtil:
                    # Tính độ hỗ trợ mong đợi
                    expected_support = self.calculate_expected_support(conditional_pattern_base, new_prefix)
            
                    # Tính giá trị tiện ích
                    utility = self.calculate_utility(conditional_pattern_base, new_prefix)
                    # xây dựng cây UHUFP cho new_prefix
                    conditional_pattern_tree = self.build_conditional_tree(conditional_pattern_base, miu)
                    # conditional_pattern_tree.print_tree()
                    # conditional_pattern_tree.print_header_table()
                    
                    # print(f"{new_prefix} : {expected_support} : {utility}")
                    
                    # kiểm tra độ hỗ trợ mong đợi với ngưỡng phổ biến và ngưỡng tiện ích
                    if expected_support > minSup:
                        if utility >= minUtil:
                            # Add the pattern to top_k list if necessary
                            top_k.append((tuple(sorted(new_prefix)), expected_support, utility))
                            # Sort and maintain top-k
                            top_k.sort(key=lambda x: (x[1], x[2]), reverse=True)
                            if len(top_k) > k:
                                top_k.pop()
                            if len(top_k) == k:
                                minSup = top_k[-1][1]
                        # tiếp tục gọi hàm với cây vừa tạo ở trên để khai thác các mẫu tiếp theo
                        if self.calculate_twu(conditional_pattern_base, new_prefix) >= minUtil:
                            if conditional_pattern_tree.root.children:
                                self.mine_top_k_patterns(conditional_pattern_tree, k, minSup, minUtil, expected_support, new_prefix, top_k)
    
        return top_k



    def collect_paths(self, uhufp_tree, prefix: List[str]) -> List[Tuple[List[Tuple[str, float, float, float]], int]]:
        """Phương thức thu thập các đường dẫn của prefix

        Args:
            uhufp_tree (UHUFP_tree): cây UHUFP của prefix
            prefix (List[str]): mẫu cần thu thập đường dẫn

        Returns:
            List[Tuple[List[Tuple[str, float, float, float]], int]]: các đường dẫn và các thông tin của đường dẫn
        """
        paths = []
        
        # Lấy item cuối cùng của prefix
        last_item = prefix[-1] if prefix else None
        # print(f"LAST ITEM: {last_item}")
        if last_item and last_item in uhufp_tree.header_table:
            for node in uhufp_tree.header_table[last_item]:
                path = []
                current_node = node
                
                # duyệt từ node hiện tại về gốc của cây
                while current_node and current_node.item is not None:
                    path.append((current_node.item, current_node.prob, current_node.utility, current_node.nu))
                    if current_node.parent is None:
                        break
                    current_node = current_node.parent
                
                # Đảo ngược path để có thứ tự từ gốc đến node hiện tại
                path.reverse()
                
                # Kiểm tra xem path có chứa tất cả các item trong prefix không
                if all(item in [p[0] for p in path] for item in prefix):
                    paths.append((path, node.count))
        # for path, count in paths:
            # print(f"Path: {path}, Count: {count}")
        return paths



    def build_conditional_pattern_base(self, uhufp_tree, item: str, min_util: float) -> List[Tuple[Dict[str, float], Dict[str, float], int]]:
        """phương thức thu thập đường dần của item và tổ chức lại đường dẫn

        Args:
            uhufp_tree (UHUFP_tree): cây UHUFP của item
            item (str): mẫu cần thu thập đường dẫn
            min_util (float): ngưỡng tiện ích

        Returns:
            List[Tuple[Dict[str, float], Dict[str, float], int]]: _description_
        """
        paths = self.collect_paths(uhufp_tree, item)
        conditional_pattern_base = []

        # print(f"Paths for item {item}: {paths}")
        path_utilities_item = defaultdict(int)
        for path, count in paths:
            path_utility = path[-1][3]  # Total utility of the path

            # tính tiện ích đường dẫn của item trong đường dẫn
            for i in path:
                path_utilities_item[i[0]] += path_utility

            # print(f"Path: {path}")
            # print(f"Path utility: {path_utility}")
            # print(f"Path utilities item: {dict(path_utilities_item)}")
        path_util_item = 0
        # tính tiện ích đường dẫn của item trong CPB
        for i in item:
            path_util_item += path_utilities_item[i]
        unpromissing_item = []
        miu = {}
        # các giá trị tiện ích nhỏ nhất của các item trong path
        for path, count in paths:
            for i, s, u, _ in path:
                if i in miu:
                    miu[i] = min(miu[i], u)
                else:
                    miu[i] = u

        for path, count in paths:
            filtered_pu = 0
            for i, s, _, u in path:
                if path_utilities_item[i] <= min_util:
                    filtered_pu += miu[i] * count
                    # print(f"Item: {i}, Node Utility: {u}, Min Utility: {miu[i]}, Count: {count}")
                    # print(f"Filtered Utility for item {i}: {filtered_pu}")
                    unpromissing_item.append((i,s,u))  
            
            reorganized_path = []

            for i, s, u, _ in path:
                if (i,s,u) not in unpromissing_item:
                    reorganized_path.append((i, s, u, path[-1][3] - filtered_pu))
            # sắp xếp danh sách path theo giá trị tiện ích đường dẫn của item trong CPB
            sorted_reorganized_path = sorted(reorganized_path, key=lambda x: path_utilities_item[x[0]], reverse=False)
            # print(f"Reorganized Path: {reorganized_path}")
            # print(f"Sorted Reorganized Path: {sorted_reorganized_path}")
        
            if sorted_reorganized_path:
                conditional_pattern_base.append((sorted_reorganized_path, count))
                # print(f"Conditional Pattern Base Entry: {conditional_pattern_base}")
        
        return conditional_pattern_base, miu, path_util_item



    def build_conditional_tree(self, conditional_pattern_base: List[Tuple[Tuple[str,float,int], int]], miu) -> 'UHUFP_tree':
        """Xây dựng cây UHUFP cho item từ CPB của item

        Args:
            conditional_pattern_base (List[Tuple[Tuple[str,float,int], int]]): chứa các đường dẫn của item
            miu (Dict[str:int]): danh sách các giá trị tiện ích nhỏ nhất của các item trong đường dẫn

        Returns:
            UHUFP_tree: cây UHUFP
        """
        
        conditional_tree = UHUFP_tree()

        for path, count in conditional_pattern_base:
            filtered_path = {item: support for item, support,_, _ in path}
            filtered_utilities = {item: util for item, _, util, _ in path}
            filtered_path_utility = {item: pu for item, _, _, pu in path}
            if filtered_path:  # nếu tồn tại path
                # print(f"Inserting transaction into conditional tree: Path: {filtered_path}, Utilities: {filtered_utilities}")
                conditional_tree.insert_reorganized_path(filtered_path, filtered_utilities, filtered_path_utility, miu)

        return conditional_tree


    def load_transactions_and_utilities(self, transaction_file: str, utility_file: str):
        """Đọc các cơ sở dữ liệu 

        Args:
            transaction_file (str): _description_
            utility_file (str): _description_

        Returns:
            Tuple[List[Dict[str, float]], List[Dict[str, float]], List[str], Dict[str, int], int]: các giá trị giao dịch chứa xác xuất và giá trị tiện ích
        """
        
        transactions = []
        utilities = []
        items = []
        dbUtil = 0
        item_twu = defaultdict(int)

        with open(transaction_file, mode='r') as tfile, open(utility_file, mode='r') as ufile:
            tlines = tfile.readlines()
            ulines = ufile.readlines()

            # Đọc file chứ dữ liệu xác xuất
            header = tlines[0].strip().split()
            items = [item.strip() for item in header]

            for line1, line2 in zip(tlines[1:], ulines):
                row = line1.strip().split()
                transaction = {items[i]: float(row[i]) for i in range(len(items)) if float(row[i]) > 0}
                transactions.append(transaction)

                parts = line2.strip().split(':')
                item_list = parts[0].split()
                utility_values = list(map(int, parts[2].split()))
                utility = {item_list[i]: utility_values[i] for i in range(len(item_list)) if item_list[i] in transaction}
                utilities.append(utility)
                dbUtil += int(parts[1])
                # print(transaction_utility)
                self.database_size += 1

            for utility in utilities:
                for item, util in utility.items():
                    item_twu[item] += sum(utility.values())
        
        return transactions, utilities, item, item_twu, dbUtil

    def build_tree_and_mine_patterns(self, transaction_file: str, utility_file: str, k: int, percentage: float, minSup: float):
        """Chạy thuật toán TUHUFP_Growth

        Args:
            transaction_file (str): các giao chứa xác xuất
            utility_file (str): giao chứa tiện ích
            k (int): số lượng mẫu cần khai thác
            percentage (float): ngưỡng tiện ích (%)
            minSup (float): ngưỡng phổ biến
        """
        
        self.start_timestamp = time.time()
        self.candidates = 0
        self.database_size = 0
        transactions, utilities, items, item_twu, dbUtil = self.load_transactions_and_utilities(transaction_file, utility_file)
        promising_items = set()
        unpromising_items = set()
        reorganized_transaction = defaultdict(float)
        reorganized_utilities = defaultdict(int)
        
        uhufp_tree = UHUFP_tree()
        minUtil = int(dbUtil*percentage)
        # lấy danh sách các item hứa hẹn
        for item, twu in item_twu.items():
            if twu >= self.minUtil:
                promising_items.add(item)
            else:
                unpromising_items.add(item)

        # print(promising_items)

        for transaction, utility in zip(transactions, utilities):
            # print(transaction)
            reorganized_transaction = {item: prob for item, prob in transaction.items() if item in promising_items}
            reorganized_utilities = {item: util for item, util in utility.items() if item in reorganized_transaction}
            sorted_items = sorted(reorganized_transaction.keys(), key=lambda item: item_twu[item], reverse=True)
            # print(sorted_items)
            # xây dựng cây UHUFP
            uhufp_tree.insert_reorganized_transaction(reorganized_transaction, reorganized_utilities, sorted_items)

        # uhufp_tree.print_header_table()
        # uhufp_tree.prune_patterns(k, item_twu, minUtil)
        # uhufp_tree.prune_tree(uhufp_tree.root)
        
        # print({'database util: ': dbUtil})
        print({'min util: ': minUtil})

        # Bắt đầu theo dõi bộ nhớ
        tracemalloc.start()
        self.top_k_UHUFP = self.mine_top_k_patterns(uhufp_tree, k, minSup, minUtil, 1.0)
        self.minUtil = minUtil
        self.top_k_UHUFP.sort(key=lambda x: x[2], reverse=True)
        # Kết thúc theo dõi bộ nhớ và lấy thông tin
        _, self.peak_memory_usage = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.peak_memory_usage /= 10**6  # Chuyển đổi sang MB

        self.end_timestamp = time.time()
        # print("\nTop-K High Utility Frequent Patterns:")
        for i, (pattern, expSup, utility) in enumerate(self.top_k_UHUFP, 1):
            print(f"{i}. Pattern: {pattern}, ExpSup: {expSup}, Utility: {utility}")

    def print_stats(self, path):
        """Thống kê thông tin kết quả chạy thuật toán

        Args:
            path (str): đường dẫn lưu thông tin
        """
        total_utility = 0
        directory = os.path.dirname(path)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        with open(path, 'w') as output_file:
            output_file.write(f"minUtil: {self.minUtil}\n")
            self.top_k_UHUFP.sort(key=lambda x: x[2], reverse=True)
            for i, (pattern, expSup, utility) in enumerate(self.top_k_UHUFP, 1):
                total_utility += utility
                output_file.write(f"{i}. Pattern: {pattern}, ExpSup: {expSup}, Utility: {utility}\n")
            output_file.write("=============  TOP-K UFPs v1.20 - STATS =============\n")
            output_file.write(f" Transactions count from database : {self.database_size}\n")
            output_file.write(f" Candidates count : {self.candidates}\n")
            output_file.write(f" Total utility : {total_utility}\n")
            output_file.write(f" Algorithm run time : {self.end_timestamp - self.start_timestamp:.0f} seconds\n")
            output_file.write(f" Peak memory usage : {self.peak_memory_usage:.2f} MB\n")  # Ghi lại peak memory usage

## Thực nghiệm

In [ ]:
transaction_file = './data/example.txt'
utility_file = './data/example_utility.txt'
k = 6
percentage = 0.3
minSup = 0.0
tuhufp_growth = TUHUFP_Growth()
tuhufp_growth.build_tree_and_mine_patterns(transaction_file, utility_file, k, percentage, minSup)
tuhufp_growth.print_stats("./out/TUHUFP_Growth/output_example.txt")

## Hiện thực foodmart

In [ ]:
transaction_file = './data/input_foodmart.txt'
utility_file = './data/foodmart_utility.txt'
k = 100
percentage = 0.0004
minSup = 0.0
tuhufp_growth = TUHUFP_Growth()
tuhufp_growth.build_tree_and_mine_patterns(transaction_file, utility_file, k, percentage, minSup)
tuhufp_growth.print_stats("./out/TUHUFP_Growth/foodmart/output_foodmart_top_100.txt")

## Hiện thực Chess

In [ ]:
transaction_file = './data/input_chess.txt'
utility_file = './data/chess_utility.txt'
k = 100
percentage = 0.001
minSup = 0.0
tuhufp_growth = TUHUFP_Growth()
tuhufp_growth.build_tree_and_mine_patterns(transaction_file, utility_file, k, percentage, minSup)
tuhufp_growth.print_stats("./out/TUHUFP_Growth/chess/output_chess_top_100.txt")

## Hiện thực retail

In [ ]:
transaction_file = '../../data/input_retail.txt'
utility_file = '../../data/retail_utility.txt'
k = 100
percentage = 0.0001
minSup = 0.0
tuhufp_growth = TUHUFP_Growth()
tuhufp_growth.build_tree_and_mine_patterns(transaction_file, utility_file, k, percentage, minSup)
tuhufp_growth.print_stats("../../out/TUHUFP_Growth/retail/output_retail_top_100.txt")

# Thuật toán 3: UHUOPM

## Thư viện

In [1]:
import sys, os
import time
import tracemalloc
import random
random.seed(42)

## Cài đặt cấu trúc dữ liệu

### PUO-lists

In [2]:
class PUO:
    """
    Là một đối tượng lưu trữ các thông tin i của mỗi giao dịch (TID, xác xuất, tỷ lệ tiện ích, tỷ lệ tiện ích còn lại)
    """
    def __init__(self, TID, prob, utility, r_utility, twu, uo, ruo):
        """
        Khởi tạo PUO

        Args:
            TID (int): mã giao dịch
            prob (float): xác xuất tồn tại
            uo (float): giá trị tiện ích
            ruo (float): tiện ích giao dịch
        """
        self.TID = TID
        self.prob = prob
        self.utility = utility
        self.r_utility = r_utility
        self.twu = twu
        self.uo = round(uo, 4)
        self.ruo = round(ruo, 4)
        self.zrutils = self.utility if self.r_utility == 0 else 0

    def __repr__(self):
        return f"PUO(TID={self.TID}, prob={self.prob:.4f}, utility={self.utility}, r_utility={self.r_utility}, twu={self.twu}, uo={self.uo:.4f}, ruo={self.ruo:.4f})\n"


In [3]:
class PUOList:
    """
    Là một đối tượng lưu trữ danh sách giao dịch của i (PUO)
    """
    def __init__(self, name=None):
        """
        Khởi tạo PUOList

        Args:
            name (str): tên của i
            PUO (PUO): PUO
        """
        self.name = name
        self.puo_list = []

    def add_puo(self, PUO):
        self.puo_list.append(PUO)

    def __repr__(self):
        return f"PUOList(Name={self.name}, PUO={self.puo_list})\n"


In [4]:
class PFUTable:
    """
    PFUTable là đối tượng chứa các thông tin của một i bao gồm: tên, tần suất, tỷ lệ tiện ích, tỷ lệ tiện ích còn lại 
    """
    def __init__(self, name=None, puo_list=None):
        """
        khởi tạo PFUTable

        Args:
            name (str): tên của cup (tức tên của i)
            puo_list (PUOList): danh sách PUOList của i
        """
        self.name = name
        self.sup = len(puo_list.puo_list) if puo_list.puo_list else 0
        self.utility = round(sum(suo_list.utility for suo_list in puo_list.puo_list) if puo_list.puo_list else 0, 2)
        self.r_utility = round(sum(suo_list.r_utility for suo_list in puo_list.puo_list) if puo_list.puo_list else 0, 2)
        self.twu = sum(suo_list.twu for suo_list in puo_list.puo_list) if puo_list.puo_list else 0
        self.pro = sum(suo_list.prob for suo_list in puo_list.puo_list) if puo_list.puo_list else 0
        self.uo = round((sum(suo_list.uo for suo_list in puo_list.puo_list) / self.sup if puo_list.puo_list else 0) , 4)
        self.ruo = round((sum(suo_list.ruo for suo_list in puo_list.puo_list) / self.sup if puo_list.puo_list else 0), 4)
        self.zrutils = round(sum(suo_list.utility for suo_list in puo_list.puo_list if suo_list.r_utility == 0) if puo_list.puo_list else 0, 2)

    def __repr__(self):
        return f"PFUTable(name={self.name}, sup={self.sup}, pro={self.pro:.4f}, utility={self.utility}, r_utility={self.r_utility}, twu={self.twu}, uo={self.uo:.4f}, ruo={self.ruo:.4f})\n"

In [5]:
class MMU:
    def __init__(self, name, MMU):
        self.name = name
        self.MMU = MMU

In [ ]:
# a b c d e
# 0.6 0 0.8 0.5 0
# 0 0.7 0.4 0 0
# 0.6 0.6 0.9 0.8 0
# 0 0.5 0 0.8 0
# 0.9 0.7 0.9 0.6 0.8
# 0 0 0.9 0 0.8
# 0 0 0.4 0.9 0
# 0.6 0.8 0 0.8 0.5
# 0.6 0 0.5 0.3 0
# 0 0 0.6 0 0.7

# a d c:65:14 7 44
# b c:37:4 33
# a b d c:38:21 4 2 11
# b d:11:8 3
# e a b d c:49:9 7 6 5 22 
# e c:58:36 22
# d c:23:1 22
# e a b d:61:36 21 2 2
# a d c:59:14 1 44
# e c:42:9 33

## Cài đặt thuật toán

In [174]:
class UHUOPM:
    def __init__(self):
        self.start_timestamp = 0
        self.end_timestamp = 0
        self.database_size = 0
        self.mmu_dict = {}
        self.puo_list_dict = {}
        self.pfu_table_dict = {}
        self.utility_max = {}
        self.item_name = []
        self.min_sup = 0
        self.min_utility = 0
        self.PHUOPs = set()
        self.peak_memory_usage = 0

    def read_data(self, file_path):
        file_paths = file_path.split(", ")
        try:
            with open(file_paths[0], 'r') as file1, open(file_paths[1], 'r') as file2:
                print("Reading data . . .")
                tlines = file1.readlines()
                ulines = file2.readlines()   
                self.database_size = len(tlines)
                return tlines, ulines   
        except FileNotFoundError as e:
            print(f"File not found: {e}")
            print("STOP ALGORITHM !!!")
            sys.exit(0)
        except Exception as e:
            print(f"An error occurred: {e}")
            sys.exit(0)
    
    def process_data(self, file_path, GLMU):
        tlines, ulines = self.read_data(file_path)

        self.is_name = tlines[0].strip().split(" ")

        for i in self.is_name:
            self.puo_list_dict[i] = PUOList(i)

        for idx_line, line in enumerate(tlines[1:]):
            probabilities = list(map(float, line.strip().split())) 

            utility = ulines[idx_line]
            parts = utility.split(":")
            keys = parts[0].split()  
            total_sum = int(parts[1]) 
            values = list(map(int, parts[2].split()))  

            result = {key: value for key, value in zip(keys, values)}
            result["sum"] = total_sum
            
            for idx, prob in enumerate(probabilities):
                if prob > 0:
                    i = self.is_name[idx]
                    if i in result.keys():
                        positions = keys.index(i)
                        utility = result[i]
                        r_utility = sum(values[positions+1:])
                        uo = result[i] / result['sum']
                        ruo = sum(values[positions+1:]) / result['sum']

                        self.puo_list_dict[i].add_puo(PUO(idx_line + 1, prob, utility, r_utility, total_sum, uo, ruo))

        for idx, i in enumerate(self.is_name):
            self.mmu_dict[i] = max((GLMU / 100) * random.randint(1, 200), GLMU)
            self.pfu_table_dict[i] = PFUTable(i, self.puo_list_dict.get(i))

    def run_UHUOPM(self, file_path, k):
        

        # sorted_by_utility = sorted(self.pfu_table_dict.values(), key=lambda x: x.utility)
        # # a = [i.name for i in pfu_sort]
        # for i in sorted_by_utility:
        #     if i not in pfu_sort and i.utility > self.min_utility:
        #         pfu_sort.append(i)
        #         pfu_sort.remove(min(pfu_sort, key=lambda x: x.utility))
        #         self.min_utility = min(pfu_sort, key=lambda x: x.utility).utility
        # # self.min_sup = min(pfu_sort, key=lambda x: x.pro).pro
        # # b = [i.name for i in pfu_sort]

        # primary = [i.name for i in pfu_sort]

        # secondary = [
        #     i.name for i in self.pfu_table_dict.values()
        #     if i.r_utility > self.min_utility
        # ]

        # primary = sorted(primary, key=lambda x: self.pfu_table_dict.get(x).pro, reverse=True)
        # secondary = sorted(secondary, key=lambda x: self.pfu_table_dict.get(x).pro, reverse=True)

        # for i in primary:
        #     self.judge(i, k)
        
        # print(primary)
        # print(secondary)


        # secondary = [i.name for i in pfu_sort]

        
        # pfu_sort = sorted(self.pfu_table_dict.values(), key=lambda x: x.pro, reverse=True)
        
        # top_k_is = pfu_sort[:k]

        # self.min_utility = min(
        #     top_k_is,  
        #     key=lambda x: x.utility
        # ).utility

        # self.is_name = [i.name for i in top_k_is]

        # self.is_name = [i.name for i in pfu_sort]
        
        # self.item_name = sorted(self.item_name, key=lambda x: self.pfu_table_dict.get(x).utility, reverse=True)
        
        # self.min_sup = min(a, key=lambda x: x.pro).pro
        # self.is_name.extend(
        #     i.name
        #     for i in pfu_sort
        #     if i.name not in self.is_name and (i.utility + i.r_utility) > self.min_utility
        # )

        # print(len(self.is_name) ,(self.is_name))

        # print(self.puo_list_dict.keys())
        # print(self.pfu_table_dict)

        self.start_timestamp = time.time()

        D = self.database_size

        if 'foodmart' in file_path:
            GLMU = (k / 100) * 1000
        elif 'chess' in file_path or 'retail' in file_path:
            GLMU = (k / 100) * 100000
        else:
            GLMU = (k / 100)

        self.process_data(file_path, GLMU)

        
        # self.min_utility = min(pfu_sort, key=lambda x: x.utility).utility

        # is_with_high_twu = [i for i in self.pfu_table_dict.values() if i.twu >= self.min_utility and self.calculate_lu('', i.name) >= self.calculate_emu('', i.name)]
        # self.item_name = [i.name for i in is_with_high_twu]

        pfu_sort1 = sorted(self.pfu_table_dict.values(), key=lambda x: x.utility, reverse=True)[:k]
        pfu_sort = sorted(self.pfu_table_dict.values(), key=lambda x: x.pro, reverse=True)[:k]
        # self.item_name = sorted(self.item_name, key=lambda x: self.pfu_table_dict.get(x).utility, reverse=True)[:k]
        
        self.min_utility = min(pfu_sort1, key=lambda x: x.utility).utility
        # self.min_sup = min(pfu_sort, key=lambda x: x.pro).pro
        self.item_name = [i.name for i in pfu_sort1 if i.pro > self.min_sup]
        self.item_name = sorted(self.item_name, key=lambda x: self.pfu_table_dict.get(x).pro, reverse=True)[:k]
        # print(self.min_sup, self.min_utility)
        for x_a in self.item_name:
            pfu_x_a = self.pfu_table_dict.get(x_a)
            if pfu_x_a.pro < self.min_sup:
                break
            
            if pfu_x_a.utility >= self.min_utility and pfu_x_a.utility > self.calculate_miu(x_a):
                if pfu_x_a.pro > self.min_sup: 
                    self.judge(x_a, k)

        tracemalloc.start()
        self.PHUOP_Search("", self.item_name, k)
        # self.PHUOP_Search("", primary, secondary, k)
        self.PHUOPs = sorted(self.PHUOPs, key=lambda x: float(x[2].split(': ')[1]), reverse=True)[:k]

        i = 1
        for pfu in self.PHUOPs:
            print(f" {i}: {pfu}")
            i +=1

        _, self.peak_memory_usage = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.peak_memory_usage /= 10**6
        self.end_timestamp = time.time()
    
    # def PHUOP_Search(self, X, primary, secondary, k):
    #     D = self.database_size
    #     for x_a in primary:
    #         pfu_x_a = self.pfu_table_dict.get(x_a)
    #         if pfu_x_a.pro < self.min_sup:
    #             return
    #         # print(x_a, pfu_x_a.utility >= self.min_utility, pfu_x_a.pro > self.min_sup)
    #         if pfu_x_a.utility > self.min_utility:
    #             if pfu_x_a.pro > self.min_sup: 
    #                 self.judge(x_a, k)

    #         if pfu_x_a.utility + pfu_x_a.r_utility >= self.min_utility:
    #             if pfu_x_a.pro > self.min_sup:
    #                 primary_extend = []
    #                 secondary_extend = []
                    
    #                 for x_b in secondary:
    #                     # print(x_b)
    #                     if self.pfu_table_dict.get(x_b).pro > self.min_sup and x_b != x_a:
    #                         puo_list = self.puo_list_dict.get(x_b)
    #                         max_prob = max(puo.prob for puo in puo_list.puo_list)
    #                         if pfu_x_a.pro * max_prob > self.min_sup:
    #                             x_ab = self.construct(X, x_a, x_b)
    #                             if x_ab != None:
    #                                 pfu_x_ab = self.pfu_table_dict.get(x_ab)
    #                                 # print(x_ab, pfu_x_ab.utility, pfu_x_ab.r_utility, self.min_sup, pfu_x_ab.pro > self.min_sup, self.calculate_lu(x_a, x_b) > self.min_utility)
    #                                 if pfu_x_ab.utility + pfu_x_ab.r_utility >= self.min_utility and pfu_x_ab.pro > self.min_sup:
    #                                     primary_extend.append(x_ab)
    #                                 if self.calculate_lu(x_a, x_b) >= self.min_utility:
    #                                     secondary_extend.append(x_ab)
    #                 # print(x_a, primary_extend, secondary_extend)
    #                 self.PHUOP_Search(x_a, primary_extend, secondary_extend, k)
    def calculate_lu(self, x_a, x_b):
        # if x_a > x_b:
        #     x_a, x_b = x_b, x_a
        if x_a == '':
            s = 0
            for i in self.puo_list_dict.get(x_b).puo_list:
                s += i.utility
            return s

        combined_utility = 0
        i_a = self.puo_list_dict.get(x_a).puo_list
        i_b = self.puo_list_dict.get(x_b).puo_list
        for Ea in i_a:
            for Eb in i_b:
                if Ea.TID == Eb.TID:
                    combined_utility += (Ea.utility + Ea.r_utility)
        
        return combined_utility
    
    def calculate_miu(self, x):
        selected_values = [self.mmu_dict[key] for key in x if key in self.mmu_dict]
        return min(selected_values) if selected_values else 0

    def calculate_smu(self, x):
        last_index = -1
        for char in x:
            if char in self.item_name:
                last_index = self.item_name.index(char)
        result = self.item_name[last_index + 1:]

        return min(self.calculate_miu(x), self.calculate_miu(result))

    def calculate_emu(self, x, e_x):
        last_index = -1
        for char in e_x:
            if char in self.item_name:
                last_index = self.item_name.index(char)
        result = self.item_name[last_index + 1:]

        if x == '':
            return min(self.calculate_miu(e_x), self.calculate_miu(result))

        return min(self.calculate_miu(x), self.calculate_miu(e_x), self.calculate_miu(result))
    
    def PHUOP_Search(self, X, exten_of_x, k):
        D = self.database_size    
        # print(X, exten_of_x)
        for idx, x_a in enumerate(exten_of_x):
            pfu_x_a = self.pfu_table_dict.get(x_a)
            
            if pfu_x_a.pro < self.min_sup:
                # print(X, x_a)
                return
                
            if pfu_x_a.utility >= self.min_utility and pfu_x_a.utility > self.calculate_miu(x_a):
                if pfu_x_a.pro > self.min_sup: 
                    self.judge(x_a, k)

            x_u_ru = pfu_x_a.utility + pfu_x_a.r_utility
            if x_u_ru - pfu_x_a.zrutils >= self.min_utility and x_u_ru > self.calculate_smu(x_a):
                if pfu_x_a.pro > self.min_sup:
                    exten = []
                    for x_b in exten_of_x[idx + 1:]:
                        if self.pfu_table_dict.get(x_b).pro > self.min_sup and self.calculate_lu(x_a, x_b) > self.calculate_emu(x_a, x_b):
                            puo_list = self.puo_list_dict.get(x_b)
                            max_prob = max(puo.prob for puo in puo_list.puo_list)
                            # print(pfu_x_a.pro * max_prob, pfu_x_a.pro, max_prob ,self.min_sup)
                            if pfu_x_a.pro * max_prob > self.min_sup:
                                x_ab = self.construct(X, x_a, x_b)
                                # print(x_a, x_b, x_ab)
                                if x_ab != None:
                                    pfu_x_ab = self.pfu_table_dict.get(x_ab)
                                    if pfu_x_ab.pro > self.min_sup and pfu_x_ab.utility + pfu_x_ab.r_utility > self.calculate_smu(x_ab) and pfu_x_ab.twu > self.min_utility:
                                        exten.append(x_ab)
                            else:
                                break
                        else:
                            break
                    # exten = sorted(exten, key=lambda x: self.pfu_table_dict.get(x).pro, reverse=True)
                    self.PHUOP_Search(x_a, exten, k)

    def judge(self, x, k):
        pfu_x = self.pfu_table_dict.get(x)
        if len(self.PHUOPs) >= k:
            self.min_sup = float(min(
                self.PHUOPs,  
                key=lambda x: float(x[1].split(': ')[1])
            )[1].split(': ')[1])

            self.min_utility = int(min(
                self.PHUOPs,  
                key=lambda x: float(x[2].split(': ')[1])
            )[2].split(': ')[1])

            min_utility_item = min(
                self.PHUOPs, 
                key=lambda x: float(x[2].split(': ')[1])
            )

            self.PHUOPs.remove(min_utility_item)
        
        self.PHUOPs.add((
            f"Name: {x}",
            f"prob: {pfu_x.pro:.2f}",
            f"utility: {pfu_x.utility}"
        ))        
        

    def construct(self, x, xa, xb):
        xa_list = xa.split(", ")
        xb_list = xb.split(", ")
        unique_numbers = set(xa_list + xb_list)
        x_ab = ", ".join(sorted(unique_numbers))
        self.puo_list_dict[x_ab] = PUOList(x_ab)
        self.pfu_table_dict[x_ab] = PFUTable(x_ab, PUOList(x_ab))
        TUTIL = self.pfu_table_dict[xa].utility + self.pfu_table_dict[xa].r_utility
        flag = False

        for Ea in self.puo_list_dict[xa].puo_list:
            for Eb in self.puo_list_dict[xb].puo_list:
                if Ea.TID == Eb.TID:
                    flag = True
                    if(self.puo_list_dict.get(x)):
                        for E in self.puo_list_dict.get(x).puo_list:
                            if E.TID == Ea.TID:
                                E_ab = PUO(Ea.TID, (Ea.prob * Eb.prob) / E.prob, Ea.utility + Eb.utility - E.utility, Eb.r_utility, Ea.twu, Ea.uo + Eb.uo - E.uo, Eb.ruo)

                                self.pfu_table_dict[x_ab].pro += E_ab.prob
                                self.pfu_table_dict[x_ab].uo += E_ab.uo
                                self.pfu_table_dict[x_ab].ruo += E_ab.ruo
                                self.pfu_table_dict[x_ab].utility += E_ab.utility
                                self.pfu_table_dict[x_ab].r_utility += E_ab.r_utility
                                self.pfu_table_dict[x_ab].twu += E_ab.twu
                                break
                    else:
                        E_ab = PUO(Ea.TID, (Ea.prob * Eb.prob), Ea.utility + Eb.utility, Eb.r_utility, Ea.twu, Ea.uo + Eb.uo, Eb.ruo)

                        self.pfu_table_dict[x_ab].pro += E_ab.prob
                        self.pfu_table_dict[x_ab].uo += E_ab.uo
                        self.pfu_table_dict[x_ab].ruo += E_ab.ruo
                        self.pfu_table_dict[x_ab].utility += E_ab.utility
                        self.pfu_table_dict[x_ab].r_utility += E_ab.r_utility
                        self.pfu_table_dict[x_ab].twu += E_ab.twu
                    
                    self.pfu_table_dict[x_ab].sup += 1
                    self.puo_list_dict[x_ab].add_puo(E_ab)
            if flag == False:
                TUTIL = TUTIL - Ea.utility - Ea.r_utility
                if TUTIL < self.calculate_smu(xa):
                    return None
            flag = False
            
        return x_ab


    def print_stats(self, path):
        """
        Thống kê thông tin kết quả chạy thuật toán

        Args:
            path (str): đường dẫn lưu thông tin
        """        
        total_utility = 0
        directory = os.path.dirname(path)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        with open(path, 'w') as output_file:
            output_file.write(f"minUtil: {self.min_utility}\n")
            for t in self.PHUOPs:
                total_utility += int(t[2].split(": ")[1])
                output_file.write(f"{t[0]}: {t[1]}: {t[2]}\n")
            output_file.write("=============  TOP-K UHOUPM - STATS =============\n")
            output_file.write(f" Transactions count from database : {self.database_size}\n")
            output_file.write(f" Sum utility : {total_utility}\n")
            output_file.write(f" Algorithm run time : {self.end_timestamp - self.start_timestamp:.0f} seconds\n")
            output_file.write(f" Peak memory usage : {self.peak_memory_usage:.2f} MB\n")  
        

In [175]:
algorithm = UHUOPM()
algorithm.run_UHUOPM("./data/example.txt, ./data/example_utility.txt", 6)
algorithm.print_stats("./out/UHUOPM/output_example.txt")

Reading data . . .
 1: ('Name: a, c, d', 'prob: 0.97', 'utility: 42')
 2: ('Name: b, c, d', 'prob: 0.87', 'utility: 40')
 3: ('Name: c, d', 'prob: 1.95', 'utility: 38')
 4: ('Name: a, d', 'prob: 1.20', 'utility: 35')
 5: ('Name: d', 'prob: 3.30', 'utility: 30')
 6: ('Name: a, b', 'prob: 1.64', 'utility: 23')


In [176]:
# chạy với foodmart
algorithm = UHUOPM()
algorithm.run_UHUOPM("./data/input_foodmart.txt, ./data/foodmart_utility.txt", 300)
algorithm.print_stats("./out/UHUOPM/foodmart/output_foodmart_top_25.txt")

Reading data . . .
 1: ('Name: 1373', 'prob: 13.55', 'utility: 25560')
 2: ('Name: 1292', 'prob: 12.60', 'utility: 24642')
 3: ('Name: 988', 'prob: 9.30', 'utility: 24187')
 4: ('Name: 225', 'prob: 8.04', 'utility: 22951')
 5: ('Name: 222', 'prob: 9.33', 'utility: 22910')
 6: ('Name: 272', 'prob: 13.54', 'utility: 22678')
 7: ('Name: 1426', 'prob: 9.96', 'utility: 21692')
 8: ('Name: 1161', 'prob: 10.54', 'utility: 21540')
 9: ('Name: 1045', 'prob: 9.48', 'utility: 21090')
 10: ('Name: 446', 'prob: 6.93', 'utility: 20625')
 11: ('Name: 83', 'prob: 6.68', 'utility: 20563')
 12: ('Name: 1518', 'prob: 7.94', 'utility: 20416')
 13: ('Name: 1180', 'prob: 8.06', 'utility: 19890')
 14: ('Name: 1012', 'prob: 11.65', 'utility: 19861')
 15: ('Name: 175', 'prob: 9.14', 'utility: 19635')
 16: ('Name: 1380', 'prob: 7.99', 'utility: 19548')
 17: ('Name: 1368', 'prob: 7.57', 'utility: 19525')
 18: ('Name: 1497', 'prob: 7.19', 'utility: 19406')
 19: ('Name: 937', 'prob: 7.24', 'utility: 19133')
 20: (

In [ ]:
# chạy với chess
algorithm = UHUOPM()
algorithm.run_UHUOPM("./data/input_chess.txt, ./data/chess_utility.txt", 25)
algorithm.print_stats("./out/UHUOPM/chess/output_chess_top_25.txt")

In [ ]:
# chạy với chess
algorithm = UHUOPM()
algorithm.run_UHUOPM("./data/input_chess.txt, ./data/chess_utility.txt", 50)
algorithm.print_stats("./out/UHUOPM/chess/output_chess_top_50.txt")

In [ ]:
# chạy với chess
algorithm = UHUOPM()
algorithm.run_UHUOPM("./data/input_chess.txt, ./data/chess_utility.txt", 75)
algorithm.print_stats("./out/UHUOPM/output_chess_top_75.txt")

In [177]:
# chạy với chess
algorithm = UHUOPM()
algorithm.run_UHUOPM("./data/input_chess.txt, ./data/chess_utility.txt", 100)
algorithm.print_stats("./out/UHUOPM/output_chess_top_100.txt")

Reading data . . .
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_2884\34182722.py", line 3, in <module>
    algorithm.run_UHUOPM("./data/input_chess.txt, ./data/chess_utility.txt", 100)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_2884\705314200.py", line 167, in run_UHUOPM
    self.PHUOP_Search("", self.item_name, k)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_2884\705314200.py", line 276, in PHUOP_Search
    if self.pfu_table_dict.get(x_b).pro > self.min_sup and self.calculate_lu(x_a, x_b) > self.calculate_emu(x_a, x_b):
                                                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_2884\705314200.py", line -1, in calculate_lu
KeyboardInterrupt

During handling of the above exception, another e

In [ ]:
# chạy với chess
algorithm = UHUOPM()
algorithm.run_UHUOPM("./data/input_chess.txt, ./data/chess_utility.txt", 125)
algorithm.print_stats("./out/UHUOPM/output_chess_top_125.txt")

In [ ]:
# chạy với chess
algorithm = UHUOPM()
algorithm.run_UHUOPM("./data/input_chess.txt, ./data/chess_utility.txt", 150)
algorithm.print_stats("./out/UHUOPM/output_chess_top_150.txt")

In [ ]:
algorithm = UHUOPM()
algorithm.run_UHUOPM("./data/input_retail.txt, ./data/retail_utility.txt", 100)
algorithm.print_stats("./out/UHUOPM/output_retail_top_100.txt")

In [ ]:
algorithm = UHUOPM()
algorithm.run_UHUOPM("./data/input_pumsb2.txt, ./data/pumsb_utility.txt", 0.0005, 0.005, 0.0005, 300)